In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.image import imread
from sklearn.metrics import classification_report,confusion_matrix

from keras import optimizers

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

import warnings
warnings.filterwarnings('ignore')

from timeit import default_timer as timer
from datetime import timedelta

In [ ]:
src_dir = os.path.join('C:\\Users\\username\\Projects\\project', 'hands')

In [ ]:
test_path = os.path.join(src_dir, 'test')
train_path = os.path.join(src_dir, 'train')

In [ ]:
dimension1 = []
dimension2 = []
for image in os.listdir(os.path.join(test_path, 'real')):
    
    img = imread(os.path.join(test_path, 'real', image))
    d1,d2,colors = img.shape
    dimension1.append(d1)
    dimension2.append(d2)

In [ ]:
np.mean(dimension1)


In [ ]:
np.mean(dimension2)

In [ ]:
img_shape = (140,130,3)

In [ ]:
image_generator = ImageDataGenerator(rotation_range=20, 
                               width_shift_range=0.10, 
                               height_shift_range=0.10, 
                               rescale=1/255, 
                               shear_range=0.1, 
                               zoom_range=0.1, 
                               horizontal_flip=True, 
                               vertical_flip=True,       
                               fill_mode='constant' 
                              )

In [ ]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(5,5),input_shape=img_shape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(5,5),input_shape=img_shape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(5,5),input_shape=img_shape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(5,5),input_shape=img_shape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())

model.add(Dense(2048))
model.add(Activation('relu'))

model.add(Dropout(0.5))

model.add(Dense(1))
model.add(Activation('sigmoid'))

opt = optimizers.Adam(learning_rate=1e-04)

model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [ ]:
model.summary() 

In [ ]:
early_stop = EarlyStopping(monitor='val_loss',patience=10)

In [ ]:
batch_size = 256

In [ ]:
train_image_gen = image_generator.flow_from_directory(train_path,
                                               target_size=img_shape[:2],
                                                color_mode='rgb',
                                               batch_size=batch_size,
                                               class_mode='binary')

In [ ]:
test_image_gen = image_generator.flow_from_directory(test_path,
                                               target_size=img_shape[:2],
                                               color_mode='rgb',
                                               batch_size=batch_size,
                                               class_mode='binary',shuffle=False)

In [ ]:
train_image_gen.class_indices

In [ ]:
start = timer()

In [ ]:
results = model.fit_generator(train_image_gen,epochs=100,
                              validation_data=test_image_gen,
                             callbacks=[early_stop])

In [ ]:
end = timer()
print(timedelta(seconds=end-start))

In [ ]:
model.save('hand_fake_detector.h5')

In [ ]:
losses = pd.DataFrame(model.history.history)

In [ ]:
losses[['loss','val_loss']].plot()

In [ ]:
model =  load_model('hand_fake_detector.h5')

In [ ]:
test_image_gen.class_indices

In [ ]:
pred_probabilities = model.predict_generator(test_image_gen)

In [ ]:
predictions = pred_probabilities > 0.5

In [ ]:
print(classification_report(test_image_gen.classes,predictions))

In [ ]:
confusion_matrix(test_image_gen.classes,predictions)